In [ ]:
import os
import polars as pl
import py_vncorenlp
import torch
from transformers import (
    AutoModel,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModel,
    Trainer,
    TrainingArguments,
    logging,
)
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

logging.set_verbosity_error()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 6
N_SPLITS = 10

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModelForSequenceClassification.from_pretrained(
    # "NlpHUST/vibert4news-base-cased", num_labels=5
    "vinai/phobert-base-v2",
    num_labels=5,
)


def seed_everything(seed_value):
    torch.manual_seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True


df = (
    pl.read_parquet(
        "hf://datasets/khangnghiem/public/transform/company_reviews/transformed_company_reviews.parquet"
    )
    .select("segmented_review", "review_rating")
    .with_columns(
        pl.col("segmented_review").list.join(" "),
        # .map_elements(tokenizer.encode, return_dtype=pl.List(pl.Int32)),
        pl.col("review_rating") - 1,
    )
)

seed_everything(86)

In [28]:
# train_X, temp_X, train_y, temp_y = train_test_split(
#     torch.tensor([(df[["segmented_review"]].map_rows(tokenizer.encode).to_series().to_list())]),
#     df["review_rating"].to_list(),
#     test_size=0.2,
#     stratify=df["review_rating"].to_list(),
# )

# val_X, test_X, val_y, test_y = train_test_split(
#     temp_X,
#     temp_y,
#     test_size=0.5,
#     stratify=temp_y,
# )
df["segmented_review"].to_list()

# tokenizer.encode(
# "Công_ty tốt Môi_trường và phúc_lợi tốt , đồng_nghiệp nhưng Không có gì để bình_luận về công_ty . Môi_trường và phúc_lợi tốt , đồng_nghiệp . Không có gì là xấu , quy_mô công_ty lớn nên khối_lượng công_việc có_thể lớn và quy_trình phức_tạp",
# )

['Công_ty tốt Môi_trường và phúc_lợi tốt , đồng_nghiệp nhưng Không có gì để bình_luận về công_ty . Môi_trường và phúc_lợi tốt , đồng_nghiệp . Không có gì là xấu , quy_mô công_ty lớn nên khối_lượng công_việc có_thể lớn và quy_trình phức_tạp',
 'Lương thấp , lãnh_đạo thiếu chuyên_nghiệp , chuyên_môn , nợ lương , chèn_ép nhân_viên . Các bạn đừng nghĩ rằng công_ty lớn , cứ khoe top 1 top 2 là ngon cơm ... Không có đâu , họ chỉ đang lôi_kéo các cổ_đông nhà_đầu_tư vào để đầu_tư thôi chứ đừng mơ đến chuyện thăng_tiến hay tăng lương trong cái công_ty gia_đình này . Công_ty dàn lãnh_đạo cấp cao toàn_bộ đều là người trong gia đình.Giám đốc mới thì non_trẻ độ 9 x ( con_nuôi của tổng giảm đốc ) năng_lực lãnh_đạo kém_cỏi đến độ nợ tiền_lương nhân_viên mấy tháng liền , chèn_ép nhân_viên nghỉ để khỏi phải đền_bù hợp_đồng . ( Nếu không tự nghỉ thì không chi_trả thưởng tháng 13 như đã giao_kèo ) Rách lắm ! Lương thì cứ cố_định 3 năm rồi không_những không tăng mà_còn cắt_giảm , lấy cớ này kia để trừ lươ

['Môi_trường tốt , cơ_hội nghề_nghiệp , phúc_lợi ấn_tượng . ( RV ) . Không . Không được làm_việc lâu_dài tại cty',
 'Công_ty không đóng BHXH mặc_dù vẫn thu , tăng ca k OT , k có 1 chế_độ nào khác .. Không . Quản_lý cũng chỉ là chức_danh thôi , chưa có trưởng phòng nào được nhận thưởng tết cả do toàn bị đuổi việc trước tết',
 'Môi_trường tốt để phát_triển .. Nhà_vệ_sinh nam của toà nhà hơi kín , ngộp . Các team có_vẻ hơi ít đi chơi với nhau nên ngoài công_việc ra mọi người chưa được thân lắm .. Đồng_đội đặc_biệt các anh_chị nhân_viên rất nhiệt_tình hỗ_trợ các bạn mới . Làm tại nhà và nghỉ_phép cá_nhân không giới_hạn . Không có OT , sau 6h hầu_như không bao_giờ bị gọi chạy nhiệm_vụ vì đã có kế_hoạch kỹ_lưỡng . Công_ty rất chú_trọng sức_khoẻ nhân_viên .',
 'Đông người . Quá đông người , chắc phải gần 2000 người , nhiêu kinh_khủng . Rất đông người , mấy bạn bên cctalk cực xinh , có máy pha cafe',
 'cách làm_việc khá là cứng_nhắc và quy_củ như thời đi học vậy , wfh cũng khó_khăn ( RV ) . - 

In [4]:
from review_sentiment import SentimentDataset


sentiment_df = SentimentDataset(df, tokenizer=tokenizer)
sentiment_df


In [ ]:
sentence = "Chúng_tôi là những nghiên_cứu_viên ."

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)
print(features.last_hidden_state.shape)


torch.Size([1, 7, 768])


In [ ]:
def train_model(model, tokenizer, train_dataset, test_dataset, model_name):
    training_args = TrainingArguments(
        output_dir=f"./results/{model_name}",
        num_train_epochs=4,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir=f"./logs/{model_name}",
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
    )

    def compute_metrics(eval_pred):
        import numpy as np
        from sklearn.metrics import accuracy_score

        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return {"accuracy": accuracy_score(labels, predictions)}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
bert_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", num_labels=5
)

bert_train = SentimentDataset(train_texts, train_labels, bert_tokenizer)
bert_test = SentimentDataset(test_texts, test_labels, bert_tokenizer)

bert_trainer = train_model(bert_model, bert_tokenizer, bert_train, bert_test, "mBERT")


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
phobert_model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base", num_labels=5
)

phobert_train = SentimentDataset(train_texts, train_labels, phobert_tokenizer)
phobert_test = SentimentDataset(test_texts, test_labels, phobert_tokenizer)

phobert_trainer = train_model(
    phobert_model, phobert_tokenizer, phobert_train, phobert_test, "PhoBERT"
)
